In [2]:
!pip install transformers

In [0]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from tensorflow.keras.callbacks import ModelCheckpoint
from google.colab import drive

In [0]:
MAX_LEN=512

In [6]:
drive.mount('/content/drive/')
print(os.listdir('./'))
os.chdir('drive/My Drive/SWM-stock-prediction/code')
#!git clone https://github.com/baani-khurana/SWM-stock-prediction.git

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
['.config', 'drive', 'sample_data']


In [7]:
os.listdir('./')

['.ipynb_checkpoints',
 'articlescraper.py',
 'metadata.csv',
 'news_join.ipynb',
 'preprocess.ipynb',
 'stockdata',
 'tweetdata',
 'unzip_data.py',
 'model_bertBCE.h5',
 'model_bert.ckpt.data-00001-of-00002',
 'model_bert.ckpt.data-00000-of-00002',
 'model_bert.ckpt.index',
 'checkpoint']

In [0]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
    
    return np.array(all_tokens)

In [0]:
def build_model(transformer, max_len=512):
  input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
  sequence_output = transformer(input_word_ids)[0]
  cls_token = sequence_output[:, 0, :]
  out = Dense(1, activation='sigmoid')(cls_token)

  model = Model(inputs=input_word_ids, outputs=out)
  model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [10]:
%%time
transformer_layer = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

CPU times: user 3.85 s, sys: 1.64 s, total: 5.49 s
Wall time: 8.16 s


In [0]:
df = pd.read_csv('../Data/final_dataset_joined_aapl240_onlyMentions.csv')

In [0]:
df = df[df.filteredtext_aapl != '[]']
del df['Unnamed: 0']
df.index = range(df.shape[0])
#df['encoded_aapl'] = df.filteredtext_aapl.apply(text_encoder, args=(tokenizer, MAX_LEN))
train_df = df[df.stock_time <= "2018-12-01 00:00:00"]
test_df = df[df.stock_time > "2018-12-01 00:00:00"]

In [0]:
#df['encoded_aapl'] = df.encoded_aapl.apply(lambda x: list(x))

In [13]:
df.head()

,site,news_file_loc,filteredtext_aapl,filteredtext_amzn,news_timestamp,stock_time,open,close,low,high,volume,label
0,businesswire.com,../Data/News/2018-01/news_0000265.json,nape summit week will also feature the annual ...,[],2017-12-07 20:00:00,2017-12-07 22:30:00,169.73,170.43,168.93,169.29,28528,1
1,yahoo.com,../Data/News/2018-01/news_0001827.json,from apples hugely anticipated iphone x to sam...,[],2017-12-08 21:37:00,2017-12-08 22:30:00,169.98,170.05,168.79,169.31,24782,1
2,yahoo.com,../Data/News/2018-01/news_0000282.json,"they are not just the largest browser, but the...",[],2017-12-12 01:55:00,2017-12-12 02:30:00,172.61,172.80,172.59,172.67,90,1
3,investingnews.com,../Data/News/2018-01/news_0001849.json,the humanitarian crisis in the drc has placed ...,[],2017-12-12 22:10:00,2017-12-12 22:30:00,171.78,172.14,171.55,171.70,21640,-1
4,seekingalpha.com,../Data/News/2018-01/news_0001855.json,cramer prefers finisar (nasdaq: fnsr ) after a...,[],2017-12-14 11:42:00,2017-12-14 18:30:00,172.41,173.12,171.63,172.14,24351,1


In [14]:
train_df.shape[0]/df.shape[0]

0.8169713665581434

In [0]:
test_df.index

Int64Index([58922, 58923, 58924, 58925, 58926, 58927, 58928, 58929, 58930,
            58931,
            ...
            72259, 72260, 72261, 72262, 72263, 72264, 72265, 72266, 72267,
            72268],
           dtype='int64', length=13347)

In [0]:
aapl_news = df.filteredtext_aapl.values
labels = df.label.values

In [0]:
'''
enc = tokenizer.encode(aapl_news[2], add_special_tokens=True)
len(enc)
for sent in aapl_news:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)'''

203

In [0]:
train_input = bert_encode(train_df.filteredtext_aapl.values, tokenizer, 2048)

In [0]:
aapl_news

array(["['\\nNAPE Summit Week will also feature the annual NAPE Summit Charities Luncheon with Tom Brokaw as keynote speaker ; a new Job Fair for upstream professionals; educational seminars presented by industry organizations AIPN, IPAA, SEG and AAPL; domestic and international NAPE Theaters running two days of Prospect Previews and Capital and Service Provider Presentations ; and a two-day Expo on nearly 15 acres of exhibition space', '\\nNAPE was founded in 1993 to help stimulate oil and gas industry activity during a downturn', '\\nThe NAPE Global Business Conference explores the most pressing topics in the domestic and international oil and gas industry each year, featuring presentations from C-level industry experts one day before the NAPE Expo opens', ' The Expo will conclude with a Summit Sweepstakes drawing for a 2018 Silver Ford F-150 in celebration of NAPEs 25 th anniversary', ' The 2018 NAPE Global Business Conference will be held February 7, 2018, and will address trends i

In [0]:
#train_input = np.array(aapl_news[train_df.index])
train_input = bert_encode(aapl_news[train_df.index], tokenizer, 512)
train_label = labels[train_df.index]
test_input = bert_encode(aapl_news[test_df.index], tokenizer, 512)
test_label = labels[test_df.index]
train_label[train_label == -1] = 0
test_label[test_label == -1] = 0

In [0]:
checkpoint_pth = '../checkpoints/bertBCE-{epoch:02d}-{val_accuracy:.2f}.ckpt'
cb = ModelCheckpoint(checkpoint_pth, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='max')
cb_list = [cb]

In [29]:
model = build_model(transformer=transformer_layer, max_len=512)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 512)]             0         
_________________________________________________________________
tf_distil_bert_model (TFDist ((None, 512, 768),)       66362880  
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 769       
Total params: 66,363,649
Trainable params: 66,363,649
Non-trainable params: 0
_________________________________________________________________


In [30]:
train_history = model.fit(
    train_input, train_label,
    validation_split=0.1,
    epochs=4,
    batch_size=8,
    verbose=1,
    callbacks=cb_list
)

Epoch 1/4
6606/6606 [==============================] - ETA: 0s - loss: 0.6766 - accuracy: 0.5507
Epoch 00001: val_accuracy improved from -inf to 0.51924, saving model to ../checkpoints/bertBCE-01-0.52.ckpt
6606/6606 [==============================] - 3549s 537ms/step - loss: 0.6766 - accuracy: 0.5507 - val_loss: 0.7025 - val_accuracy: 0.5192
Epoch 2/4
6606/6606 [==============================] - ETA: 0s - loss: 0.6240 - accuracy: 0.6280
Epoch 00002: val_accuracy improved from 0.51924 to 0.53508, saving model to ../checkpoints/bertBCE-02-0.54.ckpt
6606/6606 [==============================] - 3553s 538ms/step - loss: 0.6240 - accuracy: 0.6280 - val_loss: 0.7801 - val_accuracy: 0.5351
Epoch 3/4
6606/6606 [==============================] - ETA: 0s - loss: 0.5511 - accuracy: 0.6986
Epoch 00003: val_accuracy did not improve from 0.53508
6606/6606 [==============================] - 3552s 538ms/step - loss: 0.5511 - accuracy: 0.6986 - val_loss: 0.8396 - val_accuracy: 0.5019
Epoch 4/4
6606/6606

In [0]:
model.save_weights('../checkpoints/bertBCE_final.ckpt')

In [32]:
results = model.evaluate(test_input, test_label)
print(results)

412/412 [==============================] - 268s 650ms/step - loss: 0.8502 - accuracy: 0.5304
[0.8502473831176758, 0.5303686857223511]


In [0]:
y_pred = model.predict(test_input)

In [0]:
y_pred = np.array([1 if x >=0.5 else 0 for x in y_pred])

In [38]:
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
accuracy_score(test_label, y_pred)

0.5303686811098441

In [39]:
confusion_matrix(test_label, y_pred)

array([[3652, 3420],
       [2758, 3325]])

In [0]:
new_model = build_model(transformer_layer, max_len=512)

In [0]:
new_model.load_weights('model_bert.ckpt')

In [0]:
new_model.weights == model.weights

ValueError: ignored